# Finetuning Notebook
## LLM finetuning for AES

Code to finetune models served via Ollama for AES tasks

### Create Slim Version of test/train set

Remove metadata and just leave scores with texts

In [1]:
import pandas as pd
import os

name_dir = "data"
file_name_train = "persuade_corpus_2.0_train.csv"
file_name_test  = "persuade_corpus_2.0_test.csv"
slim_columns = ["essay_id_comp", "full_text", "holistic_essay_score"]

for split, file_name in [("train", file_name_train), ("test", file_name_test)]:
    fullpath = os.path.join(name_dir, file_name)
    print(f"loading {split} from {fullpath} …")
    df = pd.read_csv(fullpath)

    # pick only the three
    slim = df[slim_columns].copy()

    outpath = os.path.join(name_dir, f"slim_persuade_{split}.csv")
    print(f"writing slim {split} -> {outpath}")
    slim.to_csv(outpath, index=False)


loading train from data\persuade_corpus_2.0_train.csv …


C:\Users\josea\AppData\Local\Temp\ipykernel_27472\3584629915.py:12: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fullpath)


writing slim train -> data\slim_persuade_train.csv
loading test from data\persuade_corpus_2.0_test.csv …


C:\Users\josea\AppData\Local\Temp\ipykernel_27472\3584629915.py:12: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fullpath)


writing slim test -> data\slim_persuade_test.csv


Load datasets

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

llm_name  = "mistralai/Mistral-7B-v0.3"
tokenizer = AutoTokenizer.from_pretrained(llm_name, use_fast=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

raw_ds = load_dataset("csv", data_files={"train":"data/slim_persuade_train.csv",
                                     "valid":"data/slim_persuade_test.csv"})

# prompt for essay scoring
def build_prompt(text:str, score:int) -> str:
    return (
        "<s>[INST] You are an essay rater specializing in the evaluation of essays written by students from 6th to 12th grade. "
        "Read and evaluate the essay. Assign it a score from 1 to 6, in increments of 1. "
        "Your response should be only a numeric value representing the score you gave. [/INST]\n"
        f"{text.strip()}\n"
        f"</s> {score}"
    )

# tokenize and format for pytorch
def preprocess(ex):
    prompt = build_prompt(ex["full_text"], ex["holistic_essay_score"])
    tok    = tokenizer(prompt,
                       truncation=True,
                       max_length=2048,
                       padding="max_length")
    tok["labels"] = tok["input_ids"].copy()
    return tok

ds = raw_ds.map(preprocess, batched=False, remove_columns=raw_ds["train"].column_names)
ds.set_format("torch")

In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


# 4-bit quantization config
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit            = True,
    bnb_4bit_quant_type     = "nf4",
    bnb_4bit_compute_dtype  = "float16",
    bnb_4bit_use_double_quant=True
)

# load model & tokenizer
model = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=bnb_cfg,
    device_map="auto",
    # trust_remote_code=True  # Gemma needs this
)

model.config.use_cache = False
model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

# prepare PEFT (LoRA) adapter
peft_cfg = LoraConfig(
    r           = 8,
    lora_alpha  = 16,
    lora_dropout= 0.05,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_cfg)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 3,407,872 || all params: 7,251,431,424 || trainable%: 0.0470


In [3]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir            = "checkpoints",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    eval_accumulation_steps     = 1,
    learning_rate         = 3e-4,
    fp16                  = True,
    num_train_epochs      = 1,
    max_steps             = 2000,
    logging_steps         = 50,
    eval_steps            = 500,
    save_steps            = 500,
    save_total_limit      = 2,
    report_to             = "none"
)

trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = ds["train"],
    eval_dataset  = ds["valid"],
    data_collator = data_collator,
    tokenizer     = tokenizer
)

import time
from transformers import TrainerCallback

class StepLogger(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0:
            print(f"[{time.strftime('%H:%M:%S')}] Complete step {state.global_step}")

trainer.add_callback(StepLogger())

trainer.train()

C:\Users\josea\AppData\Local\Temp\ipykernel_1520\1437183572.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,2.010200
100,1.960300
150,1.955000
200,1.961200
250,1.933800
300,1.940600
350,1.877400
400,1.923800
450,1.914000
500,1.907800


[14:18:39] Complete step 100
[14:41:08] Complete step 200
[15:03:33] Complete step 300
[15:26:05] Complete step 400
[15:48:33] Complete step 500


C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[16:10:59] Complete step 600
[16:33:26] Complete step 700
[16:55:50] Complete step 800
[17:18:16] Complete step 900
[17:40:40] Complete step 1000


C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[18:02:57] Complete step 1100
[18:25:18] Complete step 1200
[18:47:35] Complete step 1300
[19:09:50] Complete step 1400
[19:32:07] Complete step 1500


C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[19:56:25] Complete step 1600
[20:20:39] Complete step 1700
[20:44:47] Complete step 1800
[21:08:52] Complete step 1900
[21:32:49] Complete step 2000


TrainOutput(global_step=2000, training_loss=1.8686896018981933, metrics={'train_runtime': 27389.3923, 'train_samples_per_second': 0.292, 'train_steps_per_second': 0.073, 'total_flos': 6.99650575171584e+17, 'train_loss': 1.8686896018981933, 'epoch': 0.046171782115360195})

### Qwen3

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

llm_name  = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(llm_name, use_fast=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

raw_ds = load_dataset("csv", data_files={"train":"data/slim_persuade_train.csv",
                                     "valid":"data/slim_persuade_test.csv"})

# prompt for essay scoring
def build_prompt(text:str, score:int) -> str:
    return (
        "<s>[INST] You are an essay rater specializing in the evaluation of essays written by students from 6th to 12th grade. "
        "Read and evaluate the essay. Assign it a score from 1 to 6, in increments of 1. "
        "Your response should be only a numeric value representing the score you gave. [/INST]\n"
        f"{text.strip()}\n"
        f"</s> {score}"
    )

# tokenize and format for pytorch
def preprocess(ex):
    prompt = build_prompt(ex["full_text"], ex["holistic_essay_score"])
    tok    = tokenizer(prompt,
                       truncation=True,
                       max_length=2048,
                       padding="max_length")
    tok["labels"] = tok["input_ids"].copy()
    return tok

ds = raw_ds.map(preprocess, batched=False, remove_columns=raw_ds["train"].column_names)
ds.set_format("torch")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Map:   0%|          | 0/173266 [00:00<?, ? examples/s]

Map:   0%|          | 0/112117 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


# 4-bit quantization config
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit            = True,
    bnb_4bit_quant_type     = "nf4",
    bnb_4bit_compute_dtype  = "float16",
    bnb_4bit_use_double_quant=True
)

# load model & tokenizer
model = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=bnb_cfg,
    device_map="auto",
    # trust_remote_code=True  # Gemma needs this
)

model.config.use_cache = False
model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

# prepare PEFT (LoRA) adapter
peft_cfg = LoraConfig(
    r           = 8,
    lora_alpha  = 16,
    lora_dropout= 0.05,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_cfg)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

trainable params: 3,833,856 || all params: 8,194,569,216 || trainable%: 0.0468


In [3]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir            = "checkpoints/qwen3",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    eval_accumulation_steps     = 1,
    learning_rate         = 3e-4,
    fp16                  = True,
    num_train_epochs      = 1,
    max_steps             = 2000,
    logging_steps         = 50,
    eval_steps            = 500,
    save_steps            = 500,
    save_total_limit      = 3,
    report_to             = "none"
)

trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = ds["train"],
    eval_dataset  = ds["valid"],
    data_collator = data_collator,
    tokenizer     = tokenizer
)

import time
from transformers import TrainerCallback

class StepLogger(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0:
            print(f"[{time.strftime('%H:%M:%S')}] Complete step {state.global_step}")

trainer.add_callback(StepLogger())

trainer.train()

C:\Users\josea\AppData\Local\Temp\ipykernel_33708\1993200313.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,2.281700
100,2.162000
150,2.162800
200,2.162300
250,2.140400
300,2.165600
350,2.084500
400,2.151300
450,2.138100
500,2.117500


[14:08:11] Complete step 100
[14:48:49] Complete step 200
[15:29:26] Complete step 300
[16:10:00] Complete step 400
[16:50:35] Complete step 500


C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[17:31:13] Complete step 600
[18:11:52] Complete step 700
[18:52:27] Complete step 800
[19:33:01] Complete step 900
[20:13:39] Complete step 1000


C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[20:54:35] Complete step 1100
[21:35:16] Complete step 1200
[22:15:49] Complete step 1300
[22:56:19] Complete step 1400
[23:37:00] Complete step 1500


C:\Users\josea\Documents\Development\StyleOverSubstance\.venv1\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[00:18:33] Complete step 1600
[01:00:08] Complete step 1700
[01:41:48] Complete step 1800
[02:23:28] Complete step 1900
[03:05:10] Complete step 2000


TrainOutput(global_step=2000, training_loss=2.085228946685791, metrics={'train_runtime': 49065.0697, 'train_samples_per_second': 0.163, 'train_steps_per_second': 0.041, 'total_flos': 7.4438141804544e+17, 'train_loss': 2.085228946685791, 'epoch': 0.046171782115360195})